In [1]:
! pip install azure-cosmos


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [26]:
#Cosmos DB imports
from azure.cosmos import CosmosClient
from azure.cosmos.aio import CosmosClient as CosmosAsyncClient
from azure.cosmos import PartitionKey, exceptions

In [31]:
! pip install --upgrade azure-cosmos
! pip show azure-cosmos



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Name: azure-cosmos
Version: 4.7.0
Summary: Microsoft Azure Cosmos Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: askdocdb@microsoft.com
License: MIT License
Location: C:\Users\Khelan Modi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages
Requires: azure-core, typing-extensions
Required-by: 


In [2]:
! pip install json
! pip install python-dotenv

! pip install openai

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
# Import the required libraries
import time
import os
import json
import uuid
from dotenv import dotenv_values
from openai import OpenAI, AzureOpenAI


#Cosmos DB imports
from azure.cosmos import CosmosClient

In [2]:
env_name = "variable.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)

cosmos_conn = config['NOSQL_PRIMARY_CONNECTION_STRING']
cosmos_key = config['NOSQL_PRIMARY_KEY']
cosmos_database = "Account"
cosmos_container = "Transactions"
nosql_uri=config["NOSQL_URI"]

openai_endpoint = config['AOAI_ENDPOINT']
openai_key = config['AOAI_KEY']
openai_api_version = config['API_VERSION']
openai_embeddings_deployment = config['AOAI_EMBEDDING_DEPLOYMENT']
openai_embeddings_model = config['AOAI_EMBEDDING_DEPLOYMENT_MODEL']
#openai_embeddings_dimensions = int(config['openai_embeddings_dimensions'])

# Create the Azure Cosmos DB for NoSQL client
cosmos_client = CosmosClient(url=nosql_uri, credential=cosmos_key)

os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
client = OpenAI()

azure_openai_embeddings = AzureOpenAI(
    api_version=openai_api_version,
    api_key= openai_key,
    azure_endpoint= openai_endpoint,
)

In [3]:
def generate_embeddings(lat_lon):
    lat_lon_str = f"{lat_lon[0]},{lat_lon[1]}"
    
    # Call OpenAI to generate embeddings (assuming text input is required)
    response = azure_openai_embeddings.embeddings.create(input=lat_lon_str, model=openai_embeddings_model)
    embeddings = response.model_dump()
    
    time.sleep(0.5)  # To avoid API rate limits
    
    return embeddings['data'][0]['embedding']

In [8]:
print(generate_embeddings("hello"))

[-0.024603435769677162, -0.007543657906353474, 0.0039630369283258915, 0.006005943287163973, 0.005148526281118393, 0.00025385376648046076, -0.006456791423261166, 0.05245941877365112, 0.009612729772925377, 0.05654925853013992, -0.004423948936164379, -0.021222073584794998, -0.02156020887196064, -0.010393663309514523, -0.017872914671897888, 0.02017546072602272, 0.024055976420640945, 0.005973740015178919, 0.015111468732357025, -0.011907225474715233, 0.010619087144732475, -0.008135396055877209, 0.0010929045965895057, 0.03503735363483429, 0.010015272535383701, 0.03126955032348633, -0.020851733162999153, -0.01722884550690651, -0.04927127808332443, -0.03374921530485153, 0.000874021730851382, 0.026342421770095825, 0.0019312006188556552, 0.019064441323280334, 0.03558481112122536, 0.007112937048077583, 0.026600049808621407, 0.004395770840346813, -0.023878857493400574, 0.022429702803492546, 0.026680557057261467, 0.008678829297423363, -0.02450682409107685, -0.0021475674584507942, -0.0016302996082231

In [4]:
# Load text-sample_w_embeddings.json which has embeddings pre-computed
data_file = open(file="data.json", mode="r") 

# OR Load text-sample.json data file. Embeddings will need to be generated using the function below.
# data_file = open(file="../../DataSet/AzureServices/text-sample.json", mode="r")

data = json.load(data_file)
data_file.close()

In [11]:
# Take a peek at one data item
print(json.dumps(data[2], indent=2))

{
  "TransactionID": "T7356",
  "Amount": 360.73,
  "Timestamp": "2024-09-15 14:04:38",
  "Location": "New York",
  "Merchant": "Amazon",
  "Fraud": false
}


In [8]:
n = 0
for item in data:
    n+=1
    item['TransactionID'] = str(n)
    amount = item["Amount"]
    timestamp = item["Timestamp"]
    location = item["Location"]
    merchant = item["Merchant"]
    location = get_city_coordinates(location)
    location_embeddings = generate_embeddings(location)
    item['locationVector'] = location_embeddings
    item['@search.action'] = 'upload'
    print("Creating embeddings for item:", n, "/" ,len(data), end='\r')

In [9]:
with open("data_w_embeddings.json", "w") as f:
    json.dump(data, f)

In [10]:
cosmos_client = CosmosClient(url=nosql_uri, credential=cosmos_key)


In [11]:
#create database
DATABASE_NAME = "fraud-nosql-db"
db= cosmos_client.create_database_if_not_exists(
    id=DATABASE_NAME
)
properties = db.read()
print(json.dumps(properties))

{"id": "fraud-nosql-db", "_rid": "IMdJAA==", "_self": "dbs/IMdJAA==/", "_etag": "\"0000f301-0000-4d00-0000-66eb19be0000\"", "_colls": "colls/", "_users": "users/", "_ts": 1726683582}


In [16]:
vector_embedding_policy = {
    "vectorEmbeddings": [
        {
            "path":"/locationVector",
            "dataType":"float32",
            "distanceFunction":"cosine",
            "dimensions":1536
        },
    ]
}

In [7]:
import azure.cosmos

azure.cosmos.__version__

'4.7.0'

In [13]:
indexing_policy = {
    "includedPaths": [
        {
            "path": "/*"
        }
    ],
    "excludedPaths": [
        {
            "path": "/\"_etag\"/?"
        },
        {
            "path": "/locationVector/*"
        }
    ],
    "vectorIndexes": [
        {"path": "/locationVector",
         "type": "quantizedFlat"
        }
    ]
}

In [17]:
from azure.cosmos import PartitionKey, exceptions
CONTAINER_NAME = "fraud-nosql-cont"
try:    
    container = db.create_container_if_not_exists(
                    id=CONTAINER_NAME,
                    partition_key=PartitionKey(path='/id', kind='Hash'),
                    indexing_policy=indexing_policy,
                    vector_embedding_policy=vector_embedding_policy)

    print('Container with id \'{0}\' created'.format(id))

except exceptions.CosmosResourceExistsError:
    print('A container with id \'{0}\' already exists'.format(id))

Container with id '<built-in function id>' created


In [22]:
with open('data_w_embeddings.json') as f:
   data = json.load(f)

container_client = db.get_container_client(CONTAINER_NAME)

for item in data:
    print("writing item ", item['TransactionID'])
    container_client.upsert_item(item)

writing item  1


CosmosHttpResponseError: (BadRequest) Message: {"Errors":["One of the specified inputs is invalid"]}
ActivityId: 30083138-beb2-4364-9fe3-36001fe309aa, Request URI: /apps/ebc9cfa3-02b3-430d-b814-8116e4c3ce71/services/8b80dd95-59df-4f1c-866c-eb502d32da42/partitions/3817f35d-04e5-4c96-8ab6-2e6ae1bf04bb/replicas/133711518027349836p, RequestStats: 
RequestStartTime: 2024-09-18T18:51:08.7756696Z, RequestEndTime: 2024-09-18T18:51:08.8080170Z,  Number of regions attempted:1
{"systemHistory":[{"dateUtc":"2024-09-18T18:50:14.5940587Z","cpu":0.515,"memory":653185804.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.0509,"availableThreads":32765,"minThreads":64,"maxThreads":32767},"numberOfOpenTcpConnection":430},{"dateUtc":"2024-09-18T18:50:24.6040651Z","cpu":0.224,"memory":652998228.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.0606,"availableThreads":32765,"minThreads":64,"maxThreads":32767},"numberOfOpenTcpConnection":430},{"dateUtc":"2024-09-

In [40]:
import numpy as np

def get_average_location_vector(container = container_client, num_purchases=10):
    """
    Retrieve the average location vector (embeddings) of the last N purchases.
    
    :param container: The Cosmos DB container object
    :param num_purchases: The number of most recent purchases to consider (default is 10)
    :return: A numpy array representing the average location vector
    """
    
    # SQL query to get the last 'num_purchases' transactions ordered by timestamp
    sql_query = """
    SELECT TOP @num_purchases c.locationVector
    FROM c
    ORDER BY c.Timestamp DESC
    """
    
    # Parameters for the query
    parameters = [
        {"name": "@num_purchases", "value": num_purchases}
    ]
    
    # Execute the query to get the location vectors
    results = container.query_items(
        query=sql_query,
        parameters=parameters,
        enable_cross_partition_query=True
    )
    
    # Collect the location vectors
    vectors = []
    for result in results:
        vectors.append(result['locationVector'])
    
    # If no vectors are found, return None
    if not vectors:
        return None
    
    # Convert the list of vectors into a numpy array
    vectors_np = np.array(vectors)
    
    # Calculate the element-wise average of the vectors
    avg_vector = np.mean(vectors_np, axis=0)
    
    return avg_vector

# Example usage
avg_location_vector = get_average_location_vector(container)
if avg_location_vector is not None:
    print(f"Average Location Vector: {avg_location_vector}")
else:
    print("No purchases found.")


Average Location Vector: [-0.0124431  -0.01449721  0.00354632 ... -0.00184411 -0.00895699
 -0.01421191]


In [56]:
def vector_search(query, current_location_vector, average_location_vector, amount, num_results=5):
    """
    Perform a vector search with filtering based on vector similarity to the query, proximity to the current location vector,
    proximity to the average location vector, and transaction amount.
    
    :param query: Text query for generating embeddings
    :param current_location_vector: The current location vector (embedding) of the current purchase
    :param average_location_vector: The average location vector (embedding) from the last N purchases
    :param amount: The amount of the current transaction
    :param num_results: Number of results to return, default is 5
    :return: A list of search results filtered by similarity to query, proximity to current and average vectors, and transaction amount
    """

    # Generate embeddings for the query
    query_embedding = generate_embeddings(query)

    if isinstance(current_location_vector, np.ndarray):
        current_location_vector = current_location_vector.tolist()
    if isinstance(average_location_vector, np.ndarray):
        average_location_vector = average_location_vector.tolist()
    # SQL query for vector similarity search combined with vector distances and amount filtering
    sql_query = """
    SELECT TOP @num_results 
        c.TransactionID,
        c.Amount, 
        c.timestamp, 
        c.location, 
        c.merchant, 
        VectorDistance(c.locationVector, @embedding) AS SimilarityScore,
        VectorDistance(c.locationVector, @average_location_vector) AS ProximityToAverage,
        VectorDistance(c.locationVector, @current_location_vector) AS ProximityToCurrent
    FROM c
    WHERE 
        VectorDistance(c.locationVector, @average_location_vector) < 0.2  -- Proximity to the average location vector
        AND VectorDistance(c.locationVector, @current_location_vector) < 0.2  -- Proximity to the current location vector
        AND c.Amount > @amount * 0.5 
        AND c.Amount < @amount * 2.0
    ORDER BY VectorDistance(c.locationVector, @embedding)
    """

    # Parameters for the SQL query
    parameters = [
        {"name": "@embedding", "value": query_embedding},  # Query embedding for similarity
        {"name": "@num_results", "value": num_results},    # Number of results
        {"name": "@average_location_vector", "value": average_location_vector},  # Use the average location vector
        {"name": "@current_location_vector", "value": current_location_vector},  # Use the current location vector
        {"name": "@amount", "value": amount}  # Transaction amount range filtering
    ]

    # Perform the query with cross-partition enabled
    results = container.query_items(
        query=sql_query,
        parameters=parameters,
        enable_cross_partition_query=True
    )

    return list(results)


In [58]:
# Assuming you already computed the average location vector and current location vector
average_location_vector = get_average_location_vector(container_client)
current_location_vector = generate_embeddings(get_city_coordinates("Boston"))

# Query parameters
query = "Recent purchase at a grocery store"
amount = 100.0  # The current transaction amount

# Perform vector search
results = vector_search(query, current_location_vector, average_location_vector, amount, num_results=5)

print(results)
# Display the results
for result in results:
    print(result)


[]


In [41]:
! pip install geopy

   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   ---------------------- ---------------- 71.7/125.4 kB 787.7 kB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB 1.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
from geopy.geocoders import Nominatim

def get_city_coordinates(city_name):
    try:
        # Create a geolocator object using Nominatim service
        geolocator = Nominatim(user_agent="MyAPP")
        
        # Geocode the city name to get location details
        location = geolocator.geocode(city_name)
        
        if location:
            # Extract the latitude and longitude from the location object
            lat = location.latitude
            lon = location.longitude
            return lat, lon
        else:
            print(f"City '{city_name}' not found.")
            return None, None
    except Exception as e:
        print(f"Error occurred: {e}")
        return None, None

